# Farkle AI

## Core Problem Definition

Consider a two-player dice game with following mechanics:

**Given:**
- 6 dice, each with values 1-6
- Set of scoring combinations C, where:
  * Single die combinations:
    - Single 1: 100 points
    - Single 5: 50 points
  * Three of a kind:
    - Three 1s: 1000 points
    - Three 2s: 200 points
    - Three 3s: 300 points
    - Three 4s: 400 points
    - Three 5s: 500 points
    - Three 6s: 600 points
  * Additional dice after three of a kind:
    - Four of a kind: 2× base value
    - Five of a kind: 4× base value
    - Six of a kind: 8× base value
  * Sequences:
    - 1-5: 500 points
    - 2-6: 750 points
    - Six consecutive numbers: 1500 points
- On each turn:
  1. Roll available dice
  2. If no valid scoring combination exists, turn ends with 0 points
  3. If valid combination exists, player must either:
     - Select one combination, set aside those dice, continue rolling with remainder
     - End turn and collect accumulated points
  4. If all dice are used, gain back all 6 dice

The objective is to find a strategy maximizing the likelihood of victory against an opponent.

## Problem Decomposition

### Base Problem P₀: Single Turn Expected Value
- Uniform probability distribution
- No cycling (ignore regaining dice)
- Maximize expected value
- Pure tree evaluation

### P₁: Add Cycling
- Add state transition: empty → 6 dice
- Introduces dependency cycle in state values
- Reference state (6 dice) determines other state values

### P₂: Points to Win
- Replace EV with probability distribution of outcomes
- Optimize for expected number of turns until win
- Changes cycling evaluation criteria

### P₃: Two Player
- Add opponent's points to win
- Both players use same optimal strategy
- Maximize probability of winning before opponent

### P₄: Non-uniform Dice
- Different probability distribution for each die
- Dice become non-fungible
- Increases state space complexity

## State Space Analysis

For each problem variant:

**P₀ State:**
- Current dice combination
- Points accumulated this turn

**P₁ State:**
- Adds cycling reference to 6-dice state

**P₂ State:**
- Dice combination
- Accumulated points
- Points needed to win

**P₃ State:**
- Dice combination
- Accumulated points
- Own points to win
- Opponent points to win

**P₄ State:**
- Individual die probabilities
- Rest same as P₃

## Solution Approach Discussion

The decomposition allows exploring each complexity layer independently:

1. P₀ requires pure expected value calculation
2. P₁ introduces fixed point computation challenge
3. P₂ transforms from EV to probability distributions
4. P₃ adds game theory considerations
5. P₄ expands state space

Key questions per level:
- P₀: Can we solve analytically?
- P₁: Is there a closed form for cyclic dependencies?
- P₂: How to efficiently represent/compute outcome distributions?
- P₃: How does optimal play change with opponent consideration?
- P₄: Can we reduce expanded state space?

---

## Solving P₀

- Number of states isn't that large. Even with non-fungible dice, 6^6 < 100,000.
    - Call state where we're choosing action with N thrown dice `ThrownState`
- In each state, I only care about the value of the best available combinations for each possible number of dice that can be put aside (state transitions)
    - TODO: `StateTransition` class that represents for which number of dice to be put aside there exists some combination, and what's the value of the best combination for that number
- Distribution of score outcomes for each `RandomState` (having N dice that have not been thrown yet) is what helps us decide in each `ThrownState`
    - TODO: `RandomState` class that represents how many dice we have. RandomState generates state transitions of all its instances and aggregates them into a single probability distribution of scores.

In [ ]:
from itertools import combinations
from collections import Counter
from typing import List, Tuple, Optional
from state_transition import StateTransition
import numpy as np

class ThrownState:
    def __init__(self, dice: List[int]):
        self.sorted_dice = dice.sort()
        self.stateTransitions = self.__getStateTransitions()

    def __getStateTransitions(self) -> List[StateTransition]:
        transitions = [None] * 6
        for i in range(len(self.sorted_dice)):
            transitions[i] = StateTransition(self.sorted_dice, i)
        return transitions


class ProbabilityDistributionNAside:
    def __init__(self):
        self.min_probability_dist_step = 50 # Minimum value difference is 50
        max_probability_dist_score = 1000 * 2 * 2 * 2 # 8,000 from throwing 6x '1'
        self.counts = np.zeros(shape=max_probability_dist_score / self.min_probability_dist_step)
    
    def addOccurence(self, value: int):
        self.counts[value / self.min_probability_dist_step] += 1

class RandomState:
    def __init__(self, n_available, value_probability_dists: List[ProbabilityDistributionNAside]):
        self.n_available = n_available
        self.value_probability_dist = value_probability_dists

    @staticmethod
    def initializeRandomStates(n_available):
        if n_available == 0:
            return None
    
